# Not Scaled Data

In [1]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [2]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [3]:
col = data[keys[0]].columns

In [4]:
def extract(list_of_texts):
    res = []
    for s in list_of_texts:
        k = 0
        for c in s:
            if c.islower():
                k += 1
        res.append(k/len(s))
    
    return pd.DataFrame({"Доля":res})

In [5]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
print(texts.keys())
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = data[key].join( extract(texts['P3:' + key1]), rsuffix = "___")

dict_keys(['P3: E', 'P3: M', 'P3: m_', 'P3: e_', "P3: I'", "P3: E'", 'P3: i_', "P3: M'", 'P3: инф', 'P3: I', 'P3: n ("надо")'])


In [6]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [7]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [8]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [9]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [10]:
# X = scale(X)
# X_bin = np.where(X <= 0, 0, 1)
X.shape

(267, 243)

## The First Union

In [11]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression, Lasso

model = Pipeline([
    #('first_scaler', StandardScaler()),
    #('first_select', SelectFromModel(LogisticRegression(C = 1, 
    #                                                    penalty="l1", 
    #                                                    multi_class = 'ovr', 
    #                                                    solver = 'liblinear'))),
    #('PolyPreproc', PolynomialFeatures(2)),
    ('second_scaler', StandardScaler()),
    ('second_select', SelectFromModel(LogisticRegression(C = 1, 
                                                        penalty="l1", 
                                                        multi_class = 'ovr', 
                                                        class_weight = "balanced",
                                                        solver = 'liblinear'))),
    ('main_model', LogisticRegression(C = 0.01,
                                      penalty = "l2",
                                      solver = "liblinear",
                                      class_weight = "balanced",
                                      multi_class = "ovr",
                                      max_iter = 100))
])

In [247]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.utils import shuffle

from sklearn.metrics import f1_score, recall_score, precision_score, make_scorer
scoring = ["f1_macro", "recall_macro", "precision_macro"]

s = 0
N = 100
full_res = {}
for key in scoring:
    full_res[key] = []
for i in range(N):
    X_, y = shuffle(X, y1)
    res = cross_validate(model, X_, y, cv = 3, scoring = scoring)
    for key in scoring:
        full_res[key].append(res['test_' + key])

KeyboardInterrupt: 

In [12]:
for key in scoring:
    print(key, np.array(full_res[key]).mean(), np.array(full_res[key]).std())

NameError: name 'scoring' is not defined

In [13]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.utils import shuffle

from sklearn.metrics import f1_score, recall_score, precision_score, make_scorer
from sklearn.model_selection import train_test_split
scoring = ["f1_macro", "recall_macro", "precision_macro"]

X_train, X_test, y_train, y_test = train_test_split(X,y1)
model.fit(X_train, y_train)
print("f1", f1_score(y_test, model.predict(X_test), average = "macro"))
print("recall", recall_score(y_test, model.predict(X_test), average = "macro"))
print("precision", precision_score(y_test, model.predict(X_test), average = "macro"))

f1 0.40229885057471265
recall 0.491725768321513
precision 0.39646464646464646


In [14]:
import shap
explainer = shap.KernelExplainer(model.predict_proba, X_train, link="logit")
shap_values = explainer.shap_values(X_test)

Using 200 background data samples could cause slower run times. Consider using shap.kmeans(data, K) to summarize the background as K weighted samples.


/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

In [18]:
shap.initjs()

In [19]:
shap.force_plot(explainer.expected_value[0], np.array(shap_values[1]))

In [24]:
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values[1], X_test)

/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/home/elias/anaconda3/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior wil

KeyboardInterrupt: 

In [23]:
keys = [20, 39, 27, 17, 33, 188, 16, 217, 24, 29, 37, 58, 50, 223, 34, 209, 30, 79, 43, 218]
for i in keys:
    print(col[i])

Доля инфинитивов
Словарь: Обсценная лексика
Доля местоимений 1 лица
Число отрицательных приставок и форм (не,ни,бес,без) / Число слов
Коэффициент логической связности
Сем. связь: LIQ
Средняя глубина синтаксического дерева
Синтакс. связь: cc
Доля глаголов 2 лица
Доля местоимений 3 лица
Словарь: Лексика положительной эмоциональной оценки
Словарь: Молодежный жаргон
Словарь: Национальные отношения
Синтакс. связь: discourse
Коэффициент опредмеченности действия (кол. глаголов / кол. существительных)
Синтакс. связь: acl:relcl
Доля местоимений 1 лица мн. числа
Сем. роль: аблатив
Словарь: Тематическая_Демография и экология
Синтакс. связь: cc:preconj


# Deep Learning Part

In [367]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y1)
y = lb.transform(y1)

In [368]:
import torch
from torch import nn
from torch.functional import F
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [421]:
class Model(nn.Module):
    def __init__(self, n_feat):
        super(Model, self).__init__()
        self.model = []
        self.model.append(nn.BatchNorm1d(n_feat))
        self.model.append(nn.Linear(in_features=n_feat, out_features=100))
        #self.model.append(nn.Dropout())
        self.model.append(nn.ReLU())
        self.model.append(nn.BatchNorm1d(100))
        #self.model.append(nn.Linear(in_features=100, out_features=100))
        #self.model.append(nn.Dropout())
        #self.model.append(nn.ReLU())
        #self.model.append(nn.BatchNorm1d(10))
        self.model.append(nn.Linear(in_features=100, out_features=3))
        self.model.append(nn.Softmax(dim = 1))
        [self.add_module(name = "final_layer%d"%(idx), module = layer) for idx, layer in enumerate(self.model)]
        

    def forward(self, whole_input):
        out = whole_input
        for layer in self.model:
            out = layer(out)
        
        return out

In [422]:
import time
history = []
s = 0
N = 10
t = time.time()
for k in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 30)
    lb.fit(y1)
    y_train = lb.transform(y_train)
    y_test = lb.transform(y_test)
    model = Model(X_train.shape[1])
    class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
    class_weight = [1/max(j,1) for j in class_weight]
    opt = torch.optim.Adam(model.parameters(), lr=1e-4)
    history = []
    for i in range(2400):
        ix = np.random.randint(0, len(X_train), 10) 
        x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
        y_batch = torch.tensor(y_train[ix], dtype=torch.float32)

        y_predicted = model(x_batch)

        loss = F.binary_cross_entropy(y_predicted, y_batch, weight = torch.tensor(class_weight, dtype=torch.float32))

        loss.backward()

        opt.step()

        opt.zero_grad()

        history.append(loss.data.numpy())
    x = torch.tensor(X_test, dtype=torch.float32)
    
    y_pred = model(x).data.numpy()
    s += f1_score(y_test.argmax(axis=1), y_pred.argmax(axis = 1), average = 'macro')
    print(f1_score(y_test.argmax(axis=1), y_pred.argmax(axis = 1), average = 'macro'))
    if k%3 == 0:
        print(k, (time.time() - t)/(k+1), s/(k+1))
s / N

0.3588039867109634
0 12.55039668083191 0.3588039867109634


/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.35062611806797855
0.3165311653116531


/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.39689922480620154
3 17.758133113384247 0.35571512372419917
0.5545454545454546
0.3214756258234519


...

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.36923076923076925


/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2742248062015504
0.3420289855072464
9 16.567649602890015 0.36384402102793434


0.36384402102793434

In [416]:
class ValModel:
    def __init__(self):
        self.model = None
        self.lb = LabelBinarizer()
        
    def fit(self, X, y):    
        X, y = X.copy(), y.copy()
        self.lb.fit(y)
        y_train = self.lb.transform(y)
        class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
        class_weight = [1/max(j,1) for j in class_weight]
        self.model = Model(X_train.shape[1])
        opt = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        history = []
        for i in range(1000):
            ix = np.random.randint(0, len(X_train), 100) 
            x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
            y_batch = torch.tensor(y_train[ix], dtype=torch.float32)

            y_predicted = self.model(x_batch)

            loss = F.binary_cross_entropy(y_predicted, y_batch, weight = torch.tensor(class_weight, dtype=torch.float32))

            loss.backward()

            opt.step()

            opt.zero_grad()

            history.append(loss.data.numpy())
        return self
            
    def predict(self, X):
        X = X.copy()
        X = torch.tensor(X, dtype = torch.float32)
        y_pred = self.model(X).data.numpy()
        return y_pred.argmax(axis = 1)

In [417]:
def my_cross_validate(model, X, y, cv, score_funcs = {}):
    X, y =   X.copy(), y.copy()
    
    X, y = shuffle(X,y)
    X_ = np.hstack([X, y.reshape(len(y), 1)])
    blocks = np.array_split(X_, cv)
    k = 0
    scores = {key:0 for key in score_funcs}
    while k < cv:
        model = ValModel()
        X_1 = blocks.pop(0)
        X_2 = np.vstack(blocks)
        X_train, y_train = X_2[:, :-1], X_2[:, -1]
        model.fit(X_train, y_train)
        X_test, y_test = X_1[:, :-1], X_1[:, -1]
        y_pred = model.predict(X_test)
        for key in score_funcs:
            s = score_funcs[key](y_pred, y_test)
            # print(key, s)
            scores[key] += s/cv
        blocks.append(X_1)
        k+=1
    return scores
    
#model = ValModel()
#X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 30)
#model.fit(X_train, y_train)
#f1_score(model.predict(X_train), y_train, average = "macro")

In [418]:
model = ValModel()

In [419]:
from sklearn.metrics import f1_score, precision_score, recall_score
score_funcs = {
    'f1_score_macro': lambda y_pred, y_test: f1_score(y_test, y_pred, average = "macro"),
    'precision': lambda y_pred, y_test: precision_score(y_test, y_pred, average = "macro"),
    'recall': lambda y_pred, y_test: recall_score(y_test, y_pred, average = "macro")
}

In [420]:
my_cross_validate(model,X, y1, 10, score_funcs = score_funcs)

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:11

{'f1_score_macro': 0.36433110546094766,
 'precision': 0.3666463749242387,
 'recall': 0.38435615554036606}